#### Here is covered: 
- Agent
- Context
- Tools

In [ ]:
from agents import Agent, Runner, function_tool, trace, RunContextWrapper
from agents.agent import ToolsToFinalOutputResult, StopAtTools
from IPython.display import display, Markdown
import requests
import random
from pydantic import BaseModel
from dataclasses import dataclass

In [ ]:
# Context creation for the agent
@dataclass
class UserContext:
    name: str
    uid: str
    is_pro_user: bool

In [ ]:
# Creating custom tools
@function_tool
def get_weather(city:str) -> str:
    """Fetch the current weather for a given city."""
    return f"The weather in {city} is sunny with a high of 25°C."

@function_tool
def get_ulka_news(topic:str) -> str:
    """Fetch the latest news from ulka"""
    res = requests.get(f"https://jsonplaceholder.typicode.com/posts/{random.randint(1, 100)}")
    output = res.json()
    return f"{output['title']} - {output['body']}"


@function_tool
def get_user_info(wrapper:RunContextWrapper[UserContext]) -> str:
    """Get User information"""
    user_info = wrapper.context
    return f"User Name: {wrapper.context.name}, User ID: {wrapper.context.uid}, Pro User: {wrapper.context.is_pro_user}"

In [ ]:
# Custom output type for the agent
class OutputType(BaseModel):
    title: str
    description: str

In [ ]:
# Agent 
agent = Agent[UserContext](
    name="Assistant",
    instructions="You are a helpful assistant",
    model="gpt-4.1-mini",
    # output_type=OutputType,
    tools=[get_user_info, get_weather, get_ulka_news],
    # tool_use_behavior="stop_on_first_tool"
    tool_use_behavior=StopAtTools(stop_at_tool_names=["get_user_info"])
    
)

In [ ]:
# Provided context value 
userinfo = UserContext(
    name="Masud Rana",
    uid="masudrana",
    is_pro_user=True
)

# Running Agent with context 
with trace("Function tools"):
    result = await Runner.run(agent, "Let me know my name, then today's weather in Hongkong, then latest news of today", context=userinfo)


In [ ]:

print(result.final_output)
